<a href="https://colab.research.google.com/github/harshagnihotri010/ai-project_ExactInference/blob/main/ai_assignment_exactInference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pgmpy

In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

~ pgmpy.models.BayesianNetwork: This imports the BayesianNetwork class from the pgmpy.models module, which is used to create Bayesian networks.

~ pgmpy.factors.discrete.TabularCPD: This imports the TabularCPD class from the pgmpy.factors.discrete module, which is used to define conditional probability distributions (CPDs).

~ pgmpy.inference.VariableElimination: This imports the VariableElimination class from the pgmpy.inference module, which provides functions for exact inference in Bayesian networks using the variable elimination algorithm.

In [ ]:
# Create the model
model = BayesianNetwork([
    ('EngineProblem', 'CarNoise'),
    ('TransmissionProblem', 'CarNoise'),
    ('ElectricalProblem', 'CarNoise'),
    ('BrakeProblem', 'CarNoise')
])

~ This line creates a Bayesian network with four nodes: EngineProblem, TransmissionProblem, ElectricalProblem, and BrakeProblem. The edges in the network indicate that CarNoise is influenced by all these problems.

In [ ]:
# Define conditional probability distributions
engine_cpd = TabularCPD(variable='EngineProblem', variable_card=2, values=[[0.3], [0.7]])
transmission_cpd = TabularCPD(variable='TransmissionProblem', variable_card=2, values=[[0.2], [0.8]])
electrical_cpd = TabularCPD(variable='ElectricalProblem', variable_card=2, values=[[0.2], [0.8]])
brake_cpd = TabularCPD(variable='BrakeProblem', variable_card=2, values=[[0.3], [0.7]])


~ These lines define the prior probabilities for each problem. For example, engine_cpd specifies that the probability of an EngineProblem is 0.3.

~ variable: Specifies the name of the variable for which the CPD is being defined.

~ variable_card: Indicates the number of possible states or values that the variable can take. In this case, all variables have a variable_card of 2, meaning they can be either 0 (false) or 1 (true).

~ values: A list of lists representing the conditional probability table. Each inner list corresponds to a specific state of the variable, and the values within the list represent the probabilities of that state given different combinations of parent variables (in this case, there are no parent variables for these prior probabilities).

In [ ]:
#values for CarNoise CPD
car_noise_cpd = TabularCPD(
    variable='CarNoise',
    variable_card=2,
    values=[
        [0.5, 0.3, 0.1, 0.1, 0.3, 0.2, 0.1, 0.05,  # P(CarNoise=0 | Parents)
         0.2, 0.1, 0.05, 0.02, 0.6, 0.4, 0.3, 0.1],  # P(CarNoise=1 | Parents)
        [0.5, 0.7, 0.9, 0.9, 0.7, 0.8, 0.9, 0.95,  # P(CarNoise=0 | Parents)
         0.8, 0.9, 0.95, 0.98, 0.4, 0.6, 0.7, 0.9]   # P(CarNoise=1 | Parents)
    ],
    evidence=['EngineProblem', 'TransmissionProblem', 'ElectricalProblem', 'BrakeProblem'],
    evidence_card=[2, 2, 2, 2]
)

~ This defines the conditional probabilities of CarNoise given different combinations of the parent variables. The values parameter is a list of lists, where each inner list represents the probabilities of CarNoise being 0 or 1 for a specific combination of parent variables.

In [ ]:
# Add CPDs to the model
model.add_cpds(engine_cpd, transmission_cpd, electrical_cpd, brake_cpd, car_noise_cpd)

~ This adds the defined CPDs to the Bayesian network.

In [ ]:
# Check if the model is valid
assert model.check_model()


~ This checks if the model is valid, ensuring that all variables have CPDs and that there are no cycles in the network.

In [ ]:
# Perform inference
inference = VariableElimination(model)
query = inference.query(
    variables=['EngineProblem', 'TransmissionProblem', 'ElectricalProblem', 'BrakeProblem'],
    evidence={'CarNoise': 1}  # Assuming car noise is present
)

~ This creates an instance of the VariableElimination class to perform inference on the model.

~ The query method is used to calculate the posterior probabilities of the specified variables (EngineProblem, TransmissionProblem, ElectricalProblem, BrakeProblem) given the evidence (CarNoise being 1).

In [ ]:
print(query)

+------------------+------------------------+----------------------+-----------------+-------------------------------------------------------------------------+
| EngineProblem    | TransmissionProblem    | ElectricalProblem    | BrakeProblem    |   phi(EngineProblem,TransmissionProblem,ElectricalProblem,BrakeProblem) |
+==================+========================+======================+=================+=========================================================================+
| EngineProblem(0) | TransmissionProblem(0) | ElectricalProblem(0) | BrakeProblem(0) |                                                                  0.0021 |
+------------------+------------------------+----------------------+-----------------+-------------------------------------------------------------------------+
| EngineProblem(0) | TransmissionProblem(0) | ElectricalProblem(0) | BrakeProblem(1) |                                                                  0.0070 |
+------------------+--------------